# Facial detection + landmark detection with Dlib

In [1]:
# import the necessary packages
from imutils import face_utils
from imutils.video import VideoStream
import numpy as np
import datetime
import time
import imutils
import dlib
import cv2

In [2]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

In [3]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

### Dlib facial detector
Instead of using **Haar cascades** built in OpenCV, we apply **Dlib's** facial detection which is based on <span style="color:red">**"HOG + Linear SVM object detector"**</span>

## Note on `imutils.VideoStream`

* Specify `usePiCamera=1` if using Rasberry Pi camera.
* Specify `usePiCamera=0` to use `self.stream = WebcamVideoStream(src=src)`

In [6]:
# initialize dlib's face detector (HOG-based) and then create a facial landmark predictor
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# initialize the video stream and warm up the camera
print("[INFO] camera sensor warming up...")
vs = VideoStream(usePiCamera=0).start()
time.sleep(2.0)

while True:
    # load image rescale and convert it to grayscale
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    # detect the face in grayscale image
    rects = detector(gray,0)
    
    for (i,rect) in enumerate(rects):
        # now each "rect" contains only one face, then we determine the facial landmarks of 
        # each face and convert tha facial landmarks into numpy array
        shape = predictor(gray,rect)
        shape = shape_to_np(shape)
        
        # conver dlib's rectangle to a OpenCV-style bounding box
        # i.e., (x,y,w,h), then draw the face bounidng box
        (x,y,w,h) = rect_to_bb(rect)
        cv2.rectangle(frame, (x,y),(x+w, y+h), (0,255,0), 2)

        # show the face number
        cv2.putText(frame, f"Face #{i+1}", (x-10, y-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0),2)

        # loop over the (x,y)-coordinates for the facial landmarks
        # and draw them on the image
        for (x,y) in shape:
            cv2.circle(frame, (x,y), 1, (0,0,255), -1)
            
            # show the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] loading facial landmark predictor...
[INFO] camera sensor warming up...


## Break down of detailed code

In [ ]:
# load image rescale and convert it to grayscale
image = vs.read()
image = imutils.resize(image, width=400)
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# detect the face in grayscale image
rects = detector(gray,1)

In the `detector()` function. The *first argument* is the gray scale image, and the *second argument* **`1`** is the **number of image pyramid layers** to apply when upscaling the image prior to applying the detector. (given bigger input image, we can increase the depth of the pyramid and the computational power increases accordingly)

## Dlib landmark detection

In [ ]:
# loop over the face detections
for (i, rect) in enumerate(rects):
    # determine the facial landmarks for the face region, then
    # convert the facial landmark (x,y)-coordinates to a NP.array
    shape = predictor(gray, rect)
    shape = shape_to_np(shape)
    
    # conver dlib's rectangle to a OpenCV-style bounding box
    # i.e., (x,y,w,h), then draw the face bounidng box
    (x,y,w,h) = rect_to_bb(rect)
    cv2.rectangle(image, (x,y),(x+w, y+h), (0,255,0), 2)
    
    # show the face number
    cv2.putText(image, f"Face #{i+1}", (x-10, y-10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0),2)
    
    # loop over the (x,y)-coordinates for the facial landmarks
    # and draw them on the image
    for (x,y) in shape:
        cv2.circle(image, (x,y), 1, (0,0,255), -1)

        
# show the output image with face detection + landmarks
cv2.imshow("output", image)
cv2.waitKey(0)
# cv2.destroyAllWindows()